<a href="https://colab.research.google.com/github/jonG312/Heat-equation-Linear-Wave-Equation-in-2D/blob/main/Heat_equation_%26_Linear_Wave_Equation_in_2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

@ Jonas Gil - UGR

# Heat equation in 2D.

In [22]:
%matplotlib inline

import numpy as np
from matplotlib import animation, rc, cm
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import pycuda.compiler as cuda_compiler
import pycuda.driver as cuda_driver
from pycuda.gpuarray import GPUArray

class LinearWaveGPU(object):
    def __init__(self):
        pass

    def initialize(self, u1, u0, c, dx, dy, dt, block_width=16, block_height=4):
        self.c = np.float32(c)
        self.dx = np.float32(dx)
        self.dy = np.float32(dy)
        self.dt = np.float32(dt)
        self.nx = np.int32(u0.shape[1]-2)
        self.ny = np.int32(u0.shape[0]-2)

        self.block_size = (block_width, block_height, 1)
        self.grid_size = (int(np.ceil(self.nx / block_width)), int(np.ceil(self.ny / block_height)), 1)
        self.stream = cuda_driver.Stream()

        self.u2_g = GPUArray(u1.shape, u1.dtype)
        self.u1_g = GPUArray(u1.shape, u1.dtype)
        self.u0_g = GPUArray(u0.shape, u0.dtype)

        self.u0_g.set_async(u0, stream=self.stream)
        self.u1_g.set_async(u1, stream=self.stream)


        cuda_kernel = """
        #define BLOCK_WIDTH %d
        #define BLOCK_HEIGHT %d

        __global__ void linearWaveEqn(float* u2, const float* u1, const float* u0, float c, float dx, float dy, float dt, int nx, int ny) {
            //Plus one to skip the "ghost cells"
            int i = blockIdx.x*blockDim.x + threadIdx.x + 1;
            int j = blockIdx.y*blockDim.y + threadIdx.y + 1;
            int tx = threadIdx.x + 1;
            int ty = threadIdx.y + 1;

            //First read into shared memory, including the local ghost cells / apron
            __shared__ float u1_shared[BLOCK_HEIGHT+2][BLOCK_WIDTH+2];
            for (int l=threadIdx.y; l<BLOCK_HEIGHT+2; l += blockDim.y) {
                int t = min(blockIdx.y*blockDim.y + l, ny);
                for (int k=threadIdx.x; k<BLOCK_WIDTH+2; k += blockDim.x) {
                    int s = min(blockIdx.x*blockDim.x + k, nx);
                    u1_shared[l][k] = u1[t*(nx+2) + s];
                }
            }
            __syncthreads();

            //Then fix the boundary conditions
            if (i == 1) {
                u1_shared[ty][tx-1] = u1_shared[ty][tx];
            }
            else if (i == nx) {
                u1_shared[ty][tx+1] = u1_shared[ty][tx];
            }
            else if (j == 1) {
                u1_shared[ty-1][tx] = u1_shared[ty][tx];
            }
            else if (j == ny) {
                u1_shared[ty+1][tx] = u1_shared[ty][tx];
            }
            __syncthreads();

            if (i >= 1 && i <= nx && j >= 1 && j <= ny) {
                u2[j*(nx+2) + i] = 2.0f*u1_shared[ty][tx] - u0[j*(nx+2)+i]
                    + c*dt/(dx*dx) * (u1_shared[ty][tx-1] - 2.0f*u1_shared[ty][tx] + u1_shared[ty][tx+1])
                    + c*dt/(dy*dy) * (u1_shared[ty-1][tx] - 2.0f*u1_shared[ty][tx] + u1_shared[ty+1][tx]);
            }
        }
        """ % (block_width, block_height)

        self.module = cuda_compiler.SourceModule(cuda_kernel, options=['--use_fast_math'])
        self.linear_wave_kernel = self.module.get_function("linearWaveEqn")
        self.linear_wave_kernel.prepare("PPPffffii")



    def step(self):
        self.linear_wave_kernel.prepared_async_call(self.grid_size, self.block_size, self.stream, \
                                                    self.u2_g.gpudata, self.u1_g.gpudata, self.u0_g.gpudata, \
                                                    self.c, self.dx, self.dy, self.dt, self.nx, self.ny)
        self.u0_g, self.u1_g, self.u2_g = self.u1_g, self.u2_g, self.u0_g


    def download(self):
        u1 = np.empty(self.u1_g.shape, self.u1_g.dtype)
        self.u1_g.get(u1)
        return u1



### Create test input data

In [23]:
c = 1.0
nx, ny = 100, 50
dx = 1.0
dy = 2.0
dt = 0.2 * min(dx / (2.0*c), dy / (2.0*c))

u0 = np.zeros((ny, nx), dtype=np.float32)
for j in range(ny):
    for i in range(nx):
        x = (i - nx/2.0) * dx
        y = (j - ny/2.0) * dy
        if (np.sqrt(x**2 + y**2) < 10*min(dx, dy)):
            u0[j, i] = 10.0
u1 = np.copy(u0)

simulator = LinearWaveGPU()
simulator.initialize(u1, u0, c, dx, dy, dt)

### Perform time integration and visualization

In [24]:
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')
X, Y = np.linspace(0, nx*dx, nx), np.linspace(0, ny*dy, ny)
X, Y = np.meshgrid(X, Y)
surf = ax.plot_surface(X, Y, u1, cmap=cm.coolwarm, linewidth=0, antialiased=True, vmin=-5, vmax=5)
ax.set_zlim(-5, 10)

def animate(i):
    print(".", end='', flush=True)
    if (i > 0):
        for k in range(10):
            simulator.step()

    u2 = simulator.download()

    ax.clear()
    surf = ax.plot_surface(X, Y, u2, cmap=cm.coolwarm, linewidth=0, antialiased=True, vmin=-5, vmax=5)
    ax.set_zlim(-5, 10)

plt.rcParams["animation.html"] = "html5"
anim = animation.FuncAnimation(fig, animate, range(30), interval=100)
plt.close()
anim

...............................

## Linear Wave Equation in 2D

In [25]:
import numpy as np
import pycuda.compiler as cuda_compiler
import pycuda.driver as cuda_driver
from pycuda.gpuarray import GPUArray

class LinearWaveGPU(object):
    def __init__(self):
        pass

    def initialize(self, u1, u0, c, dx, dy, dt, block_width=16, block_height=4):
        self.c = np.float32(c)
        self.dx = np.float32(dx)
        self.dy = np.float32(dy)
        self.dt = np.float32(dt)
        self.nx = np.int32(u0.shape[1]-2)
        self.ny = np.int32(u0.shape[0]-2)

        self.block_size = (block_width, block_height, 1)
        self.grid_size = (int(np.ceil(self.nx / block_width)), int(np.ceil(self.ny / block_height)), 1)
        self.stream = cuda_driver.Stream()

        self.u2_g = GPUArray(u1.shape, u1.dtype)
        self.u1_g = GPUArray(u1.shape, u1.dtype)
        self.u0_g = GPUArray(u0.shape, u0.dtype)

        self.u0_g.set_async(u0, stream=self.stream)
        self.u1_g.set_async(u1, stream=self.stream)


        cuda_kernel = """
        #define BLOCK_WIDTH %d
        #define BLOCK_HEIGHT %d

        __global__ void linearWaveEqn(float* u2, const float* u1, const float* u0, float c, float dx, float dy, float dt, int nx, int ny) {
            int i = blockIdx.x*blockDim.x + threadIdx.x + 1;
            int j = blockIdx.y*blockDim.y + threadIdx.y + 1;

            if (i >= 1 && i <= nx && j >= 1 && j <= ny) {
                u2[j*(nx+2) + i] = 2.0f*u1[j*(nx+2) + i] - u0[j*(nx+2) + i] + c*c*dt*dt/(dx*dx) * (u1[(j+1)*(nx+2) + i] - 2.0f*u1[j*(nx+2) + i] + u1[(j-1)*(nx+2) + i]) + c*c*dt*dt/(dy*dy) * (u1[j*(nx+2) + i+1] - 2.0f*u1[j*(nx+2) + i] + u1[j*(nx+2) + i-1]);
            }
        }
        """ % (block_width, block_height)

        self.module = cuda_compiler.SourceModule(cuda_kernel, options=['--use_fast_math'])
        self.linear_wave_kernel = self.module.get_function("linearWaveEqn")
        self.linear_wave_kernel.prepare("PPPffffii")



    def step(self):
        self.linear_wave_kernel.prepared_async_call(self.grid_size, self.block_size, self.stream, \
                                                    self.u2_g.gpudata, self.u1_g.gpudata, self.u0_g.gpudata, \
                                                    self.c, self.dx, self.dy, self.dt, self.nx, self.ny)
        self.u0_g, self.u1_g, self.u2_g = self.u1_g, self.u2_g, self.u0_g


    def download(self):
        u1 = np.empty(self.u1_g.shape, self.u1_g.dtype)
        self.u1_g.get(u1)
        return u1


### Create test input data

In [26]:
c = 1.0
nx, ny = 100, 50
dx = 1.0
dy = 2.0
dt = 0.2 * min(dx / (2.0*c), dy / (2.0*c))

u0 = np.zeros((ny, nx), dtype=np.float32)
for j in range(ny):
    for i in range(nx):
        x = (i - nx/2.0) * dx
        y = (j - ny/2.0) * dy
        if (np.sqrt(x**2 + y**2) < 10*min(dx, dy)):
            u0[j, i] = 10.0
u1 = np.copy(u0)

simulator = LinearWaveGPU()
simulator.initialize(u1, u0, c, dx, dy, dt)

### Perform time integration and visualization

In [17]:
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')
X, Y = np.linspace(0, nx*dx, nx), np.linspace(0, ny*dy, ny)
X, Y = np.meshgrid(X, Y)
surf = ax.plot_surface(X, Y, u1, cmap=cm.coolwarm, linewidth=0, antialiased=True, vmin=-5, vmax=5)
ax.set_zlim(-5, 10)

def animate(i):
    print(".", end='', flush=True)
    if (i > 0):
        for k in range(10):
            simulator.step()

    u2 = simulator.download()

    ax.clear()
    surf = ax.plot_surface(X, Y, u2, cmap=cm.coolwarm, linewidth=0, antialiased=True, vmin=-5, vmax=5)
    ax.set_zlim(-5, 10)

plt.rcParams["animation.html"] = "html5"
anim = animation.FuncAnimation(fig, animate, range(30), interval=100)
plt.close()
anim

...............................